In [1]:
import pandas as pd
import warnings
import re
import tables
import mysql.connector
warnings.filterwarnings('ignore')

In [2]:
def abre_conexao_banco_de_dados():
  global mydb
  mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

In [3]:
def retorna_tabelas_banco_de_dados(nomeTabela):
  if nomeTabela =='genealogia_embrapa':
    sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
  else:
    sql = f"SELECT * FROM {nomeTabela}"
  df =  pd.read_sql(sql, mydb)
  return df

In [4]:
def cria_df_com_somente_os_dados_novos(planilhaAntiga, planilhaNova):
  dados_em_comum = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
  dados_em_comum = dados_em_comum.drop(dados_em_comum.filter(regex='_drop$').columns, axis=1)
  dados_em_comum = dados_em_comum.drop_duplicates()
  df_merge = pd.merge(dados_2023, dados_em_comum, how='left', indicator=True, suffixes=('', '_drop'))
  # filtra somente as linhas que estão no dataframe da esquerda
  df_dados_novos = df_merge[df_merge['_merge'] == 'left_only']
  # remove a coluna "_merge"
  df_dados_novos.drop('_merge', axis=1, inplace=True)
  df_dados_novos = df_dados_novos.drop(df_merge.filter(regex='_drop$').columns, axis=1)
  df_dados_novos = df_dados_novos.drop_duplicates()
  return df_dados_novos

In [5]:
def adiciona_coluna_gs_aos_dados_novos(df,df_pedigree ):
  df_temporaria = pd.merge(df, df_pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
  df=df_temporaria
  nome_antigo = 'SEXO'
  nome_novo = 'sexo'
  indice_coluna = df.columns.get_loc(nome_antigo)
  df.rename(columns={nome_antigo: nome_novo}, inplace=True)
  return df

In [6]:
def retira_acentos_espacos(df):
  # remoção dos hifens e espaços em branco da coluna RGVACA
  df.loc[df['COMPRAC'] == 11, 'RGVACA'] = df.loc[df['COMPRAC'] == 11, 'RGVACA'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSMAE'] == 11, 'RGMAE'] = df.loc[df['GSMAE'] == 11, 'RGMAE'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSPAI'] == 11, 'RGPAI'] = df.loc[df['GSPAI'] == 11, 'RGPAI'].str.replace('-', '').str.replace(' ', '')
  return df

In [7]:
def adiciona_colunas_codprod_codpai_cod_mae(df, genealogia_abcz):
  sub_tabela_gs_0 = df[df['gs'] == 0]
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  df_temporaria1 = pd.merge(sub_tabela_gs_0, genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
  sub_tabela_gs_0=df_temporaria1
  sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()

  df_temporaria4 = pd.merge(df, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
  df=df_temporaria4
  df = df.drop_duplicates()
  return df

In [8]:
def pegar_codpai_codmae_abcz_rg_3_letras(df, genealogia_abcz):

  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela para maes
  subtabela_mae = df[df['GSMAE'] == 11]
  #e o cod_mae e nulo
  subtabela_mae
  # selecionar apenas as linhas que começam com letras
  subtabela_mae = subtabela_mae[subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Buscando cod_mae na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna cod_prod_mae no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela em que gspai = 11
  subtabela_pai = df[df['GSPAI'] == 11]

  # selecionar apenas as linhas que começam com letras
  subtabela_pai = subtabela_pai[subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na ABCZ
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Criando coluna do cod_prod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando valores da cod_pai e cod_mae do df com os valores das colunas inseridas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')

  return df


In [9]:
def pegar_codpai_codmae_abcz_rg_diferente_3_letras(df, genealogia_abcz):
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela
  subtabela_pai = df[df['GSPAI'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_pai = subtabela_pai[~subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_pai_gen'})
  subtabela_pai = subtabela_pai.drop_duplicates()
  genealogia_abcz = genealogia_abcz.drop_duplicates()
  subtabela_pai['nome4caracteres'] = subtabela_pai['nomep'].str.slice(start=0, stop=4)
  genealogia_abcz['nome4caracteres'] = genealogia_abcz['nome'].str.slice(start=0, stop=4)
  genealogia_abcz_pai = genealogia_abcz
  genealogia_abcz_pai = genealogia_abcz[genealogia_abcz['sexo']== 'M']
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz_pai[['RGVACA', 'nome4caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela mae
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = df[df['GSMAE'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_mae = subtabela_mae[~subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_mae na abcz
  subtabela_mae['nome4caracteres'] = subtabela_mae['nomem'].str.slice(start=0, stop=4)
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
  #criando coluna com os 10 caracteres do nome
  genealogia_abcz_mae = genealogia_abcz
  genealogia_abcz_mae = genealogia_abcz[genealogia_abcz['sexo']== 'F']
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz_mae[['RGVACA','nome4caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_mae_gen no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando cod_pai e cod_mae com base nas novas colunas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')
  return df

In [10]:
def adiciona_coluna_reb(df, criadores):
  criadores = criadores.rename(columns={'codgiro':'reb_giro'})
  df = pd.merge(df, criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
  df = df.drop_duplicates()
  return df

In [11]:
def calcula_dias_de_diferenca_entre_partos(df, data_2022):
  df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df, data_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
  df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
  #df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
  df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
  df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
  df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
  return df_diferenca_entre_partos_novos_e_do_banco
  #df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

In [12]:
def verifica_se_ha_id_animal_planilha_antiga(df, df_antigo):
  df_animais_com_ID = pd.merge(df, df_antigo, on =['IDANIMAL'], suffixes=('', '_drop'))
  df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
  df_animais_com_ID = df_animais_com_ID.drop_duplicates()
  return df_animais_com_ID

In [13]:
def gera_df_animais_sem_id_planilha_antiga(df, planilha_antiga):
  df_animais_sem_id = pd.merge(df, planilha_antiga, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
  df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
  df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
  return df_animais_sem_id

In [14]:
def adiciona_id_produto_nos_animais_com_id(df, genealogia_embrapa):
  df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
  df = pd.merge(df, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
  df = df.drop_duplicates()
  df = df.drop(columns='_merge')
  return df

In [15]:
def adiciona_animais_sem_id_que_possuem_id_na_gen_embrapa_ao_df_animais_com_id(animais_sem_id, df_animais_id, genealogia_embrapa):
  ids_encontrados = pd.merge(animais_sem_id, genealogia_embrapa['IDANIMAL'], left_on=['IDANIMAL'], right_on=['IDANIMAL'], how='left', suffixes=('', '_drop'), indicator=True)
  ids_encontrados = ids_encontrados.drop_duplicates()
  ids_encontrados = ids_encontrados.drop(ids_encontrados.filter(regex='_drop$').columns.difference(['IDANIMAL_drop']), axis=1)
  ids_encontrados = ids_encontrados[ids_encontrados['_merge'] == 'both']

  # dropa as colunas que não são necessárias
  ids_encontrados = ids_encontrados.drop(columns='_merge')
  df_animais_id = pd.concat([df_animais_id, ids_encontrados])
  df_animais_id = df_animais_id.drop_duplicates()
  return df_animais_id

In [16]:
def atualiza_animais_sem_id(animais_sem_id, df_animais_id):

  #Tirando o sufixo da tabela

  animais_sem_id = pd.merge(animais_sem_id, df_animais_id[['IDANIMAL', 'id_produto']], on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
  animais_sem_id = animais_sem_id[animais_sem_id['_merge']=='left_only']
  #animais_sem_id = animais_sem_id.drop(animais_sem_id.filter(regex='_drop$').columns, axis=1)
  # Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"

  animais_sem_id = animais_sem_id.drop(columns='_merge')
  return animais_sem_id

In [17]:
def buscar_ids_tabela_producao(sem_id, producao):
  producao['parto'] = producao['parto'].astype(str)
  producao['AP'] = producao['parto'].str.slice(start=0, stop=4)
  producao['MP'] = producao['parto'].str.slice(start=5, stop=7)
  #Pegando os 10 primeiros caracteres de nome dos dataframes aniamis sem id e producao
  producao['nome10caracteres'] = producao['nome'].str.slice(start=0, stop=11)
  sem_id['nome10caracteres'] = sem_id['nomea'].str.slice(start=0, stop=11)
  #removendo os espacos em branco das celulas
  producao['nome10caracteres'] = producao['nome10caracteres'].str.replace('\s+', '', regex=True)
  sem_id['nome10caracteres'] = sem_id['nome10caracteres'].str.replace('\s+', '', regex=True)

  sem_id = sem_id.drop(sem_id.filter(regex='_drop$').columns, axis=1)
  producao = producao.drop(producao.filter(regex='_drop$').columns, axis=1)

  producao['AP'] = producao['AP'].astype(int)
  producao['MP'] = producao['MP'].astype(int)
  sem_id['AP'] = sem_id['AP'].astype(int)
  sem_id['MP'] = sem_id['MP'].astype(int)
  id_encontrado_na_producao = pd.merge(sem_id, producao[['reb','AP', 'MP', 'nome10caracteres','id_produto']], on=['reb', 'AP', 'MP', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  id_encontrado_na_producao = id_encontrado_na_producao[id_encontrado_na_producao['_merge'] == 'both']
  id_encontrado_na_producao = id_encontrado_na_producao.drop(columns={'_merge', 'id_produto'})
  id_encontrado_na_producao = id_encontrado_na_producao.rename(columns={'id_produto_drop': 'id_produto'})
  return id_encontrado_na_producao

In [18]:
def adiciona_animais_encontrados_na_producao_aos_animais_com_id(animais_id, animais_encontrados_producao):
  animais_id = pd.concat([animais_id, animais_encontrados_producao])
  return animais_id

In [19]:
def procura_animais_comprac11_rg_3_na_genealogia_emb(animais_sem_id, genealogia_embrapa):
  subtabela_animais_sem_id = animais_sem_id[animais_sem_id['COMPRAC']==11]
  # selecionar apenas as linhas que começam com letras
  subtabela_animais_sem_id = subtabela_animais_sem_id[subtabela_animais_sem_id['RGVACA'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Tirando o sufixo da tabela
  df_genealogia_embrapa = genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
  df_subtabela_genealogia_emb = genealogia_embrapa[genealogia_embrapa['gs']==0]
  df_subtabela_genealogia_emb

  df_animais_encontrados_subtabela_genealogia = pd.merge(subtabela_animais_sem_id,df_subtabela_genealogia_emb[['id_produto', 'registro']], left_on=['RGVACA'], right_on=['registro'], how='left', indicator=True)
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia[df_animais_encontrados_subtabela_genealogia['_merge'] == 'both']
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='_merge')
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='registro')
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop_duplicates()
  return df_animais_encontrados_subtabela_genealogia

In [49]:
def procura_animais_comprac_dif_11_rg_3_na_genealogia_emb(animais_sem_id, genealogia_embrapa):
  subtabela_animais_sem_id = animais_sem_id[animais_sem_id['COMPRAC']!=11]
  # selecionar apenas as linhas que começam com letras
  subtabela_animais_sem_id = subtabela_animais_sem_id[subtabela_animais_sem_id['RGVACA'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Tirando o sufixo da tabela
  genealogia_embrapa = genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
  df_subtabela_genealogia_emb = genealogia_embrapa[genealogia_embrapa['gs']!=0]
  df_subtabela_genealogia_emb
  df_subtabela_genealogia_emb['nome4caracteres'] = df_subtabela_genealogia_emb['nome'].str.slice(start=0, stop=4)
  subtabela_animais_sem_id['nome4caracteres'] = subtabela_animais_sem_id['nomea'].str.slice(start=0, stop=4)

  df_animais_encontrados_subtabela_genealogia = pd.merge(subtabela_animais_sem_id,df_subtabela_genealogia_emb[['id_produto', 'registro', 'nome4caracteres']], left_on=['RGVACA', 'nome4caracteres'], right_on=['registro', 'nome4caracteres'], how='left', indicator=True)
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia[df_animais_encontrados_subtabela_genealogia['_merge'] == 'both']
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='_merge')
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop(columns='registro')
  df_animais_encontrados_subtabela_genealogia = df_animais_encontrados_subtabela_genealogia.drop_duplicates()
  return df_animais_encontrados_subtabela_genealogia

In [21]:
def adiciona_animais_encontrados_na_genealogia_aos_animais_com_id(animais_id, animais_encontrados_genealogia):
  animais_id = pd.concat([animais_id, animais_encontrados_genealogia])

  animais_id['id_produto'].fillna(animais_id['id_produto_x'], inplace=True)
  animais_id['id_produto'].fillna(animais_id['id_produto_y'], inplace=True)
  animais_id = animais_id.drop(columns='id_produto_x')
  animais_id = animais_id.drop(columns='id_produto_y')
  animais_id = animais_id.drop_duplicates()

  return animais_id

In [22]:
def busca_por_campos_personalizados_genealogia_embrapa(animais_sem_id, genealogia_embrapa):
  #Retirando sufixo tabela genealogia
  genealogia_embrapa = genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
  #Pegando ano nascimento nas colunas
  genealogia_embrapa['nasc_est'] = genealogia_embrapa['nasc_est'].astype(str)
  genealogia_embrapa['AN'] = genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
  animais_sem_id['nasc'] = animais_sem_id['nasc'].astype(str)
  animais_sem_id['AN'] = animais_sem_id['nasc'].str.slice(start=0, stop=4)
  #Criando coluna com os 10 caracteres do nome na genealogia_embrapa
  genealogia_embrapa['nome10caracteres'] = genealogia_embrapa['nome'].str.slice(start=0, stop=11)
  #Convertendo coluna ano e mes para tipo inteiro
  #genealogia_embrapa['AN'] = genealogia_embrapa['AN'].astype(int)
  #Limpando tabelas
  animais_sem_id = animais_sem_id.drop(animais_sem_id.filter(regex='_drop$').columns, axis=1)
  genealogia_embrapa = genealogia_embrapa.drop(genealogia_embrapa.filter(regex='_drop$').columns, axis=1)
  #criando dataframe com os ids_encontrados
  df_animais_com_id_na_genealogia = pd.merge(animais_sem_id, genealogia_embrapa[['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs','RGMAE', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
  df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
  df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(columns={'_merge', 'id_produto', 'registro_mae', 'registro_pai'})
  df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.rename(columns={'id_produto_drop': 'id_produto'})
  df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(df_animais_com_id_na_genealogia.filter(regex='_drop$').columns, axis=1)
  return df_animais_com_id_na_genealogia

In [23]:
def busca_ampla_genealogia_embrapa(animais_sem_id, genealogia_embrapa):

  genealogia_embrapa['nasc_est'] = genealogia_embrapa['nasc_est'].astype(str)
  genealogia_embrapa['AN'] = genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
  animais_sem_id['nasc'] = animais_sem_id['nasc'].astype(str)
  animais_sem_id['AN'] = animais_sem_id['nasc'].str.slice(start=0, stop=4)
  #Criando coluna com os 10 caracteres do nome na genealogia_embrapa
  genealogia_embrapa['nome10caracteres'] = genealogia_embrapa['nome'].str.slice(start=0, stop=11)
  df_animais_com_id_na_genealogia = pd.merge(animais_sem_id, genealogia_embrapa[['nome10caracteres','gs', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
  df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
  return df_animais_com_id_na_genealogia

In [24]:
def salvar_arquivos():
  df_fazendas.to_hdf('df_fazendas.h5', key='df', mode='w')
  df_criadores.to_hdf('df_criadores.h5', key='df', mode='w')
  df_genealogia_abcz.to_hdf('df_genealogia_abcz.h5', key='df', mode='w')
  df_animais_sem_id.to_hdf('df_animais_sem_id.h5', key='df', mode='w')
  df_animais_com_id.to_hdf('df_animais_com_id.h5', key='df', mode='w')
  df_genealogia_embrapa.to_hdf('df_genealogia_embrapa.h5', key='df', mode='w')

In [25]:
abre_conexao_banco_de_dados()
df_genealogia_embrapa = retorna_tabelas_banco_de_dados('genealogia_embrapa')
df_genealogia_embrapa =df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = retorna_tabelas_banco_de_dados('genealogia_abcz')
df_criadores = retorna_tabelas_banco_de_dados('criadores')
df_producao = retorna_tabelas_banco_de_dados('producao_embrapa')
df_fazendas = retorna_tabelas_banco_de_dados('fazendas')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [26]:
df_dados_novos = cria_df_com_somente_os_dados_novos(dados_2022, dados_2023)

In [27]:
df_dados_novos = adiciona_coluna_gs_aos_dados_novos(df_dados_novos, pedigree)

In [28]:
df_dados_novos = retira_acentos_espacos(df_dados_novos)

In [29]:
df_dados_novos =adiciona_colunas_codprod_codpai_cod_mae(df_dados_novos, df_genealogia_abcz)

In [30]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_3_letras(df_dados_novos, df_genealogia_abcz)

In [31]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_diferente_3_letras(df_dados_novos, df_genealogia_abcz)

In [32]:
df_dados_novos = adiciona_coluna_reb(df_dados_novos, df_criadores)

In [33]:
dif_partos = calcula_dias_de_diferenca_entre_partos(df_dados_novos, dados_2022)

In [34]:
df_animais_com_id = verifica_se_ha_id_animal_planilha_antiga(df_dados_novos, dados_2022)

In [35]:
df_animais_sem_id = gera_df_animais_sem_id_planilha_antiga(df_dados_novos, dados_2022)

In [36]:
df_animais_com_id = adiciona_id_produto_nos_animais_com_id(df_animais_com_id, df_genealogia_embrapa)

In [37]:
df_animais_com_id = adiciona_animais_sem_id_que_possuem_id_na_gen_embrapa_ao_df_animais_com_id(df_animais_sem_id, df_animais_com_id, df_genealogia_embrapa)

In [38]:
df_animais_sem_id = atualiza_animais_sem_id(df_animais_sem_id, df_animais_com_id)

In [39]:
lista_animais_encontrados_producao = buscar_ids_tabela_producao(df_animais_sem_id, df_producao)

In [40]:
df_animais_com_id = adiciona_animais_encontrados_na_producao_aos_animais_com_id(df_animais_com_id, lista_animais_encontrados_producao)

In [41]:
df_animais_sem_id = atualiza_animais_sem_id(df_animais_sem_id, df_animais_com_id)

In [42]:
lista_animais_encontrados_genealogia = procura_animais_comprac11_rg_3_na_genealogia_emb(df_animais_sem_id, df_genealogia_embrapa)
lista_animais_encontrados_genealogia

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto_x,nome10caracteres,id_produto_drop,id_produto_y
2,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,0,F,17054525.0,4176523.0,13961623.0,5192-GIRO,NaN,URUPIANGAF,NaN,RRP8772
3,5192,PRENDA FIV DE BRAS.,11,RRP8191,GENGIS KHAN DE BRASILIA,11,RRP6097,FERVURA FIV DE BRASILIA,11,RRP5998,...,0,F,14926361.0,9265878.0,8720772.0,5192-GIRO,NaN,PRENDAFIV,NaN,RRP8191
5,22541,FELICE FIV CABO VERDE,11,JCVL3334,JAGUAR TE DO GAVIAO,11,GAV291,ZILAH FIV CABO VERDE,11,JCVL1624,...,0,F,17198554.0,4176523.0,13865301.0,NaN,NaN,FELICEFIV,NaN,JCVL3334
7,42583,BARILOCHE KENYO,11,KOK844,ELDORADO FIV KENYO,11,KOK96,IZABELLE FIV KENYO,11,KOK526,...,0,F,14672175.0,9451056.0,12324993.0,NaN,NaN,BARILOCHEK,NaN,KOK844
9,5192,RAGIN FIV DE BRAS.,11,RRP8395,JAGUAR TE DO GAVIAO,11,GAV291,ISCA DE BRASILIA,11,RRP6699,...,0,F,15421321.0,4176523.0,11066847.0,5192-GIRO,NaN,RAGINFIVD,NaN,RRP8395
10,5192,RUANA DE BRAS.,11,RRP8430,BACO DA RTPA,11,NLT130,LALA DE BRAS.,11,RRP7119,...,0,F,15438291.0,12480405.0,12161575.0,5192-GIRO,NaN,RUANADEBR,NaN,RRP8430
11,5192,LIZ TEATRO,11,RARS10,TEATRO DA SILVANIA,11,EFC383,RENUNCIA FIV DE BRAS.,11,RRP8249,...,0,F,17112475.0,3540404.0,15128061.0,5192-GIRO,NaN,LIZTEATRO,NaN,RARS10
12,42583,MILENA KENYO,11,KOK846,NOBRE TE CAL,11,CAL4397,JARANA FIV KENYO,11,KOK625,...,0,F,14673091.0,2989503.0,12894710.0,NaN,NaN,MILENAKENY,NaN,KOK846
17,5192,REDE FIV DE BRAS.,11,RRP8488,GENGIS KHAN DE BRASILIA,11,RRP6097,DELI DE BRAS.,11,RRP5562,...,0,F,15617034.0,9265878.0,6293655.0,5192-GIRO,NaN,REDEFIVDE,NaN,RRP8488
18,5192,RENUNCIA FIV DE BRAS.,11,RRP8249,FARDO FIV F.MUTUM,11,MUT697,ESFERA DE BRASILIA TN2 DE BRAS,11,RRP7530,...,0,F,15128061.0,8530023.0,13305655.0,5192-GIRO,NaN,RENUNCIAFI,NaN,RRP8249


In [43]:
df_animais_com_id = adiciona_animais_encontrados_na_genealogia_aos_animais_com_id(df_animais_com_id, lista_animais_encontrados_genealogia)

In [44]:
df_animais_sem_id = atualiza_animais_sem_id(df_animais_sem_id, df_animais_com_id)

In [51]:
df_animais_sem_id
df_animais_sem_id.loc[df_animais_sem_id['IDANIMAL'] == 2038826]

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto,nome10caracteres,id_produto_drop,id_produto_drop
59,2381,FERTILIDADE ALTACEO JM NOVO HO,58,4899-AZ,RICKLAND ALTACEO-ET,10,AX138598,BALELA YANK JM NOVO HORIZONTE,38,3619-U,...,5,F,NaN,NaN,NaN,31300-ABCZ,NaN,FERTILIDADE,NaN,NaN


In [54]:
df_genealogia_embrapa

,id_produto,id,abcz,registro,nome,sexo,paternidade,ck_sire,id_touro,id_vaca,...,gr_tp,rebc2_cri,rebc2_pr,cat,ficha_a,cod_prod,IDANIMAL,origem,registro_pai,registro_mae
0,0001,0001,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
1,0003,0003,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
2,0006,0006,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
3,02418-249,02418-249,None,2418,nan,F,None,None,108115,108615,...,0,99999,99999,None,0,0,0,OUTRAS,A1343,18935
4,03,03,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463543,ZZZ7,ZZZ7,S,ZZZ7,AMBICAO,F,None,None,KCA472,135177,...,0,7145-ABCZ 1,11958-ABCZ 1,PO,1351780,3158020,0,ABCGIL,KCA472,KB429
463544,ZZZ8,ZZZ8,S,ZZZ8,AGASALHO,M,None,None,121118,131985,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,3247323,0,ABCZ,K830,KA7368
463545,ZZZ9,ZZZ9,S,ZZZ9,BACABAL,M,None,None,133175,178922,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,3247322,0,ABCZ,K974,KA9422
463546,ZZZA1,ZZZA1,S,ZZZA1,ANITA,F,None,None,EFC383,JDRB874,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,9499094,0,ABCZ,EFC383,JDRB874


In [50]:
lista_animais_encontrados_genealogia = procura_animais_comprac_dif_11_rg_3_na_genealogia_emb(df_animais_sem_id, df_genealogia_embrapa)
lista_animais_encontrados_genealogia

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,cod_prod,cod_pai,cod_mae,reb,id_produto_x,nome10caracteres,id_produto_drop,id_produto_drop,nome4caracteres,id_produto_y


In [ ]:
lista_animais_encontrados_genealogia_busca_personalizada = busca_por_campos_personalizados_genealogia_embrapa(df_animais_sem_id, df_genealogia_embrapa)

In [ ]:
lista_animais_encontrados_genealogia_busca_ampla = busca_ampla_genealogia_embrapa(df_animais_sem_id, df_genealogia_embrapa)

In [ ]:
#salvar_arquivos()